# Limpieza de datos y preprocesado de textos

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import unicodedata,string

PATH="/home/enriqueotero/datasets/nlp/corpusCriticasCine/"

def clean(x):
   x = unicodedata.normalize('NFKD', x).encode('ascii','ignore').lower()
   replace_punctuation = string.maketrans(string.punctuation, ' '*len(string.punctuation))
   x = x.translate(replace_punctuation)
   x = re.sub('@%$&[\n/:!,;)()_?¿¡<>]', ' ', x)
   x = re.sub(' - ', ' ', x)
   x = re.sub(' +',' ', x).strip()
   return x

def parseFile(filename):
    
    try:
        data = open(filename,'r').read()
        soup = BeautifulSoup(data, "html.parser")
        author = soup.find("review")["author"]
        title = soup.find("review")["title"]
        review_text = soup.body.get_text()
        letters_only = clean(review_text)
        result = ','.join([clean(author), clean(title), clean(review_text)])

    except:
        result = []
        
    return result

parseFile(PATH + str(999) + ".xml")

'radon,the queen,bajo mi modesta opinion me resulta una pelicula casi documental a cerca de los acontecimientos sucedidos en la casa real inglesa a tenor de la muerte de lady di y muy especialmente visto desde el punto de vista de la riena isabel de inglaterra la pelicula es entretenida e interesante sobre todo por lo reciente de los acontecimientos pero cuando termino mi sensacion fue mas la de una intencion por parte del creador de la pelicula de reconciliar la opinion publica con la imagen de la monarquia francamente lo tomare como una vision romantica de lo que pudo pasar por la mente de la monarca tras la muerte de lady di y no como lo que realmente sucediera en cualquier caso me quedo con la excepcional interpretacion de helen mirren actriz que me era completamente desconocida hasta esta pelicula asi como el parecido fisico de la reina es asombroso queda muy lejos el reflejo del principe carlos en el actor que lo interpreta alex jennings de espaldas si que da el pego pero visto d

In [2]:
N_DOCS = 1000
mydict = {i:parseFile(PATH+str(i)+".xml") for i in range(N_DOCS)}
df = pd.DataFrame(data=mydict.items(), columns=['id', 'text'])
df['author'] = df['text'].str.split(',').str[0]
df['title'] = df['text'].str.split(',').str[1]
df['text'] = df['text'].str.split(',').str[2]

df = df[df["text"] != ""].dropna()

df.to_csv(PATH+"clean_reviews.csv", index=False)
df.head()

,id,text,author,title
2,2,cada vez me gusta menos el cine de masas las p...,torbe,la guerra de los mundos
3,3,el otro dia fui a ver la de los sioux como dec...,torbe,stars wars iii la venganza de los sith
4,4,es que no la cagan en ninguna todas las pelis ...,torbe,los increibles
5,5,es un dolor esto del cine yo ya voy con miedo ...,torbe,spiderman 2
6,6,tras una insufrible primera media hora la casa...,victor trujillo,la casa de cera


# Representación Bag of Words con TF-IDF

In [3]:
#corpus = ['This is the first document.','This is the second second document.','And the third one.','Is this the first document?']
corpus = df["text"].tolist()

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer=u'word', use_idf=True, max_df=0.3, lowercase=False)
#vectorizer = TfidfVectorizer(analyzer=u'word', use_idf=True)
tfidf_matrix = vectorizer.fit_transform(corpus)
features = vectorizer.get_feature_names()

# Extracción de palabras clave

In [5]:
import operator
import numpy as np

def getKeywords(myvector, top_n, features):
    myweights = myvector.toarray().flatten()
    indices = np.argsort(myweights)[::-1]
    top_features = [features[i] for i in indices[:top_n]]
    return top_features 

N_KEYWORDS = 10
def keywords(myid):
    return getKeywords(tfidf_matrix[myid:myid+1], N_KEYWORDS, features)

print keywords(1)

[u'jedi', u'jedis', u'rollo', u'anaquino', u'palpatino', u'katana', u'jedai', u'samurais', u'peinado', u'batallitas']


In [6]:
df["keywords"] = map(keywords, range(len(df)))

In [7]:
df.head()

,id,text,author,title,keywords
2,2,cada vez me gusta menos el cine de masas las p...,torbe,la guerra de los mundos,"[peli, puta, chorradas, cruise, vieja, huir, o..."
3,3,el otro dia fui a ver la de los sioux como dec...,torbe,stars wars iii la venganza de los sith,"[jedi, jedis, rollo, anaquino, palpatino, kata..."
4,4,es que no la cagan en ninguna todas las pelis ...,torbe,los increibles,"[pelis, yankis, madre, repetirse, pa, hacen, c..."
5,5,es un dolor esto del cine yo ya voy con miedo ...,torbe,spiderman 2,"[spiderman, tele, peli, puta, mico, pastora, v..."
6,6,tras una insufrible primera media hora la casa...,victor trujillo,la casa de cera,"[cera, acojonante, peli, risa, gore, casa, com..."


# Clustering

In [8]:
from sklearn.cluster import KMeans

num_clusters = 100
clustering = KMeans(n_clusters=num_clusters)
%time clustering.fit(tfidf_matrix)

clusters = clustering.labels_.tolist()

CPU times: user 18.6 s, sys: 76 ms, total: 18.7 s
Wall time: 18.7 s


In [9]:
df["cluster"] = clusters
df.head()

,id,text,author,title,keywords,cluster
2,2,cada vez me gusta menos el cine de masas las p...,torbe,la guerra de los mundos,"[peli, puta, chorradas, cruise, vieja, huir, o...",35
3,3,el otro dia fui a ver la de los sioux como dec...,torbe,stars wars iii la venganza de los sith,"[jedi, jedis, rollo, anaquino, palpatino, kata...",35
4,4,es que no la cagan en ninguna todas las pelis ...,torbe,los increibles,"[pelis, yankis, madre, repetirse, pa, hacen, c...",35
5,5,es un dolor esto del cine yo ya voy con miedo ...,torbe,spiderman 2,"[spiderman, tele, peli, puta, mico, pastora, v...",35
6,6,tras una insufrible primera media hora la casa...,victor trujillo,la casa de cera,"[cera, acojonante, peli, risa, gore, casa, com...",74


In [10]:
df[df["cluster"] == 4]

,id,text,author,title,keywords,cluster
56,56,bueno el tito chinchan en calidad de ganador d...,tito chinchan,los tres entierros de melquiades estrada,"[tommy, drama, negro, subtitulos, resaltar, ex...",4
244,244,con pase de prensa a estrenar entre a ver en h...,ivan sainz pardo,el senor de la guerra,"[yuri, negocio, reconocemos, victima, financia...",4
367,367,simple entretenimiento protagonizado por will ...,rafa ferrer,pasado de vueltas,"[7o, ferrell, sacha, sabran, baron, motor, coh...",4
427,427,pasado de vueltas tallega nights the ballad of...,kiko de espana,pasado de vueltas,"[ferrell, vueltas, poquito, ybipolarmente, mck...",4
483,483,keenen ivory wayans salto a la fama por el ine...,oscar cantero,pequeno pero maton,"[humor, triquinuelas, botin, acogen, birriosa,...",4
731,731,la verdad es que comercialmente el cartel es u...,tito chinchan,clerks 2,"[sensato, tonterias, humor, cartel, particular...",4
796,796,el personaje del ano es borat sagdiyev un entr...,kiko de espana,borat,"[borat, eeu, puta, descojona, avergonzarian, a...",4
797,797,a los americanos les encantan los paletos el c...,jose contreras,borat,"[paletos, paleto, borat, americano, entrevista...",4
815,815,imprescindible ver esta pelicula en ingles con...,ivan sainz pardo,borat,"[borat, boca, humor, documental, subtitulos, c...",4
820,820,cada poco sale una lumbrera que nos seduce a t...,emilio calvo de mora,borat,"[borat, fea, productos, cultura, americana, mi...",4


In [12]:
df[df["cluster"] == 1]

,id,text,author,title,keywords,cluster
153,153,el argumento de 16 calles es impecable para te...,jose contreras,16 calles,"[muchacho, detective, pregunta, delincuente, w...",1
184,184,adoro las peliculas de venganzas quiza sean mi...,pablo,el caso slevin,"[recaiga, capo, narrar, mafioso, amigo, asilo,...",1
241,241,creia que lo habia visto todo creia que ningun...,caty,ultravioleta,"[ultraviolet, wimmer, equilibrium, teoria, nin...",1
291,291,bueno me apetecia ver el retorno de michael do...,rafa ferrer,la sombra de la sospecha,"[douglas, pantalla, cierto, chupado, bassinger...",1
402,402,en los anos ochenta fueron pocos los que se re...,oscar cantero,corrupcion en miami,"[jonson, don, policias, motivaron, aclarar, fa...",1
408,408,hubiese preferido que michael mann artifice de...,yul b,corrupcion en miami,"[miami, mann, farrel, forrados, corrupcion, na...",1
409,409,la esperada y deseada adaptacion al cine de la...,lady winter,corrupcion en miami,"[vice, tosar, miami, policias, tubbs, sonny, e...",1
417,417,desde que el atraco protagonizado por de niro ...,jesus lens,corrupcion en miami,"[miami, mann, vice, sonny, collateral, corrupc...",1
426,426,esta ola de revivalismo que nos recorre tiene ...,emilio calvo de mora,corrupcion en miami,"[miami, vice, caja, mann, tiros, lanchas, pand...",1
434,434,dirigida por michael mann esa simple frase ya ...,red stovall,corrupcion en miami,"[mann, miami, corrupcion, sonny, farrell, foxx...",1
